In [ ]:
import sys
sys.path.append('/home/redacted/dev/moral-summarization')

from moral_summarization.data_utils import *

results_dir = '../results/final_prompts'
extra_metrics = []
model = 'Meta-Llama-3-70B-Instruct'
seed = '345'

article_list = 'articles_in_test_set.txt'
with open(article_list) as f:
    article_list = f.read().splitlines()

In [5]:
article_name = [article_name.split('/')[-1] for article_name in article_list]
dataset = [article_name.split('/')[0] for article_name in article_list]

summaries_df = pd.DataFrame(columns=['dataset', 'article_text']+prompt_types, index=article_name)

for article, dataset in zip(article_name, dataset):
    summaries_df.loc[article, 'dataset'] = dataset

    article_text = load_article(article, dataset)
    summaries_df.loc[article, 'article_text'] = article_text

    article_path = os.path.join(results_dir, dataset, article)

    # Get all files with responses
    responses_files = [file_path for file_path in os.listdir(article_path) \
                        if 'response' in file_path]

    for prompt_type in prompt_types:
        # get all the files with this model and this prompt style
        response_files = [file_path for file_path in responses_files \
                            if model in file_path \
                            and prompt_type in file_path \
                            and seed in file_path]
        assert len(response_files) == 1, f'Expected 1 response file, got {len(response_files)}'
        response_file = response_files[0]

        response_path = os.path.join(article_path, response_file)
        summary_text = get_summary_from_response(response_path)
        summaries_df.loc[article, prompt_type] = summary_text

In [7]:
summaries_df.to_csv(f'../results/{model}_{seed}_summaries.csv', index=True)